## Jeu de données
Un labo avec des projets, des services et des personnes

In [ ]:
import json 
with open('data/lab2000.json','r') as f:
    lab = json.load(f)

# Liste des projets
lab['projects']

In [ ]:
# Liste des services
lab['services']

In [ ]:
# Liste des personnes (les 3 premières)
print(json.dumps(lab['persons'][1:3], indent=4, sort_keys=True))

## Configuration


In [ ]:
demoConfig = {
    'url': 'localhost',
    'port': '7687',
    'user': 'neo4j',
    'password': 'neo4j',
}

driverUrl = f"bolt://{demoConfig['user']}:{demoConfig['password']}@{demoConfig['url']}:{demoConfig['port']}"

In [ ]:
from scripts.draw_graph import draw_query
from py2neo import Graph

drawGraph = Graph()

# Neo4j - Python

## Jouons avec le driver officiel neo4j python

Le documentation est ici (version 1.7)

https://neo4j.com/docs/api/python-driver/1.7-preview/

## Concepts de base

- Un driver gère toutes les interactions avec la base
- Ces interactions se déroulent dans le cadre d'une session fournie par le driver
- les modifications se déroule au sein d'une transaction fournie par la session

```
driver        session        transaction
 |               |               |               
 | session()     |               |
 |-------------->|               |
 |               |   begin()     |               
 |               |-------------->|               
 |               |               |               
 |               |               |----+            
 |               |               |   write something
 |               |               |<---+           
 |               |               |               
 |               |               |----+           
 |               |               |  commit()     
 |               |               |<---+           
 |               |               |               
 |               |               |----+
 |               |               | close()              
 |               |               |<---+               
 |               |               |               
```



## Interactions avec la base
* Instancier un driver
* Obtenir une session

In [ ]:
from neo4j.v1 import GraphDatabase

driver = GraphDatabase.driver('bolt://localhost:7687')

## Modification
* Ouverture d'une transaction pour modifier
 * modification
 * commit ou rollback

### Création des noeuds

D'une façon conventionelle, puis d'une façon plus pythonique.

In [ ]:
def create_nodes_classic_way(label, data, driver):
    
    # -- Obtention session --
    session = driver.session()
    
    # -- Obention transaction --
    tx = session.begin_transaction()
    
    for name in data:
        tx.run('CREATE (:'+label+' {name : {name}})', name=name)
    
    #-- commit --
    tx.commit()
    # -- fermeture de la transaction. Elle ne peut être réutilisée.
    tx.close()

In [ ]:
create_nodes_classic_way('Project', lab['projects'], driver)

In [ ]:
def create_nodes_pythonic_way(label, data, driver):
    
    # - mode auto-commit
    
    with driver.session() as local_session:
        for name in data:
            print(name)
            # -- pas de transaction, directement au niveau de la session
            local_session.run('CREATE (:'+label+' {name : {name}})', name=name)

In [ ]:
create_nodes_pythonic_way('Service', lab['services'],driver)

### Lecture seule

Un simple `Session.run(query)` suffit.

In [ ]:
# - session globale pour le notebook
global_session = driver.session()

Les résultats sont des `Records` à parcourir.

In [ ]:
[r for r in global_session.run('MATCH (n) RETURN labels(n)[0], n.name')]

In [ ]:
global_session.run('MATCH (n:Project) RETURN count(n) as nb').single().value()

## Résultats en tant qu'entités Graph
Pour obtenir les résultats sous forme de 
* Node
* Relationship
* Path

il faut utiliser la vue `Graph` des `StatementResult`.

Au sein de graphes, il est possible d'avoir les noeuds `.nodes` et les relations `.relationships`.


In [ ]:
[r for r in global_session.run('MATCH (n:Project) RETURN n').graph().nodes]

In [ ]:
draw_query(drawGraph,'driver_project','MATCH (n:Project) RETURN n')

### Création de relations

**(:Person)-\[:MEMBER_OF\]->(:Service)**

In [ ]:
query ='''
MATCH (s:Service {name: {service}}) 
CREATE (p:Person {name: {name}})
CREATE (p)-[:MEMBER_OF]->(s)'''

with driver.session() as session:
    for p in lab['persons']:
        session.run(query, name=p['name'],service=p['service'])

In [ ]:
draw_query(drawGraph,'driver_member_of_service','MATCH p=(q:Person)-[r]->(s:Service) RETURN p')

**(:Person)-\[:WORK_IN \{function=$function}\]->(:Project)**

In [ ]:
subquery = '''
MATCH (pers:Person {name: {name}}),(project:Project {name: {project}})
MERGE (pers)-[:WORK_IN{function: {function}} ]->(project)'''
        
with driver.session() as session:
    for person in lab['persons']:        
        for project in person['projects']:
            session.run(subquery,
                       name=person['name'],
                       project=project['name'],
                       function=project['function'])

In [ ]:
draw_query(drawGraph,'driver_work_in_project',
           'MATCH p=(q:Person)-[r]->(s:Project) RETURN p')

In [ ]:
with driver.session() as session:
    records = session.run('MATCH (n)-[r:WORK_IN]->(p) return n,r,p')
    
# - les 5 premiers
for relation in list(records.graph().relationships)[0:5]:
    print(relation.items())
    print(relation.type)
    print(relation.start_node)
    print(relation.end_node)
    print('---')

### La liste des personnes qui ne sont pas associées à un projet

In [ ]:
with driver.session() as session:
    query= '''MATCH (p:Person)-[:MEMBER_OF]->(s) 
    WHERE NOT (p)-[:WORK_IN]->(:Project) 
    RETURN p.name AS name, s.name AS service'''
    records = session.run(query)
    
print(f'Statement {records.summary().statement} after {records.summary().result_available_after} ms' )

[ f"Person : {r['name']}, Service: {r['service']}" for r in records]


# Avec py2no

py2neo est un wrapper autour du driver neo4j.

Il simplifie la vie et s'intégre très bien avec plusieurs librairies communes Python

In [ ]:
from py2neo import Graph, Node, Relationship

graph = Graph()

### Avec Pandas

In [ ]:
query= '''MATCH (p:Person)-[:MEMBER_OF]->(s) 
WHERE NOT (p)-[:WORK_IN]->(:Project) 
RETURN p.name AS name, s.name AS service'''

graph.run(query).to_data_frame()

### Avec Numpy

In [ ]:
graph.run(query).to_ndarray()

## Manipulations Graphiques (pour les gourmets)

py2neo permet de manipuler le graphe directement en terme de Node, Relationship.

Le graphe est modifié localement avant de mettre à jour la base à travers des _push_ et des _pull_.

_La documentation manque singulièrement de clarté._

In [ ]:
# - Création de la ville d'orsay
city = Node('City', name='Orsay')

# - merge en matchant sur le label 'City' et la clef 'name'
graph.merge(city, 'City','name')

In [ ]:
# - recherche du noeud (roxane)
roxanne = graph.nodes.match("Person", name='Roxanne Brochu').first()

print(roxanne, type(roxanne))

# - mise en relation avec la ville
liveIn = Relationship(roxanne, "lIVE_IN", city)

# - CREATION de la relation
graph.create(liveIn)


In [ ]:
# - Recherche du service informatique

serviceInfo = graph.nodes.match('Service', name='Informatique').first()

# - recherche de la relation entre carine et le service informatique

roxanneToInformatique = graph.match((roxanne,), r_type='MEMBER_OF').first()

# - Mise à jour locale

roxanneToInformatique['function'] = 'chef'

# - mise à jour remote 
graph.push(roxanneToInformatique)

In [ ]:
graph.match((roxanne,), r_type='MEMBER_OF').first()

## Mettre à jour le graphe du projet  'Tigre'

In [ ]:
import random

# - récupération duprojet 'tigre'
tigerProject = graph.nodes.match('Project', name='Tigre').first()

# - récupération des relations (p)->[r:WORK_IN]->(:Project {name: 'Tigre'})
relations =list(graph.relationships.match((None,tigerProject),'WORK_IN')) 

# - Mise à jour d'une charge de travail aléatoire comme en vrai ? :)
for rel in relations:
    rel['workload'] = random.randint(1,8)*10


for rel in graph.relationships.match((None,tigerProject),'WORK_IN'):
    print(rel.start_node, rel['workload'])

### Regroupement des objet sous forme de Subgraph (conteneur)

In [ ]:
from py2neo import Subgraph

subgraph=Subgraph(relationships=relations)
graph.push(subgraph)

In [ ]:
for rel in graph.relationships.match((None,tigerProject),'WORK_IN'):
    print(rel.start_node, rel['workload'])

## Procédure de nettoyage


In [ ]:
import ipywidgets as widgets
from ipywidgets import interact

checkbox = widgets.ToggleButton(
    value=False,
    description='DELETE ALL',
    disabled=False
)

def delete_all(b):
    if b:
        with driver.session() as session:
            session.run('MATCH (n) DETACH DELETE n')
    
            
interact(delete_all, b=checkbox)